### Setting up the data 

In [3]:
import pandas as pd

nba_data = pd.read_csv(r"seasons_clean.csv")

In [6]:
nba_data.info()

# Selecting columns to keep/drop for data set
keep_list = [25,7,0,1,3,5,6,8,9,10,11,12,13,15,17,18,20,21,23,24,26,27,28,34]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 36 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Season  6134 non-null   int64  
 1   Player  6134 non-null   object 
 2   Pos     6134 non-null   object 
 3   Age     6134 non-null   float64
 4   Tm      6134 non-null   object 
 5   G       6134 non-null   float64
 6   GS      6134 non-null   float64
 7   MPG     6134 non-null   float64
 8   PPG     6134 non-null   float64
 9   RPG     6134 non-null   float64
 10  APG     6134 non-null   float64
 11  SPG     6134 non-null   float64
 12  BPG     6134 non-null   float64
 13  TOPG    6134 non-null   float64
 14  eFG%    6134 non-null   float64
 15  FG      6134 non-null   float64
 16  FGA     6134 non-null   float64
 17  FG%     6134 non-null   float64
 18  3P      6134 non-null   float64
 19  3PA     6134 non-null   float64
 20  3P%     6134 non-null   float64
 21  FT      6134 non-null   float64
 22  

In [11]:
### Select Player 

# Selecting a random player from test data set
X_player = X_test.sample(n=1)

print(X_player)

       PER       MPG   Age     G   GS       PPG      RPG       APG       SPG  \
2606  11.9  7.901639  29.0  61.0  0.0  2.311475  1.47541  0.131148  0.147541   

           BPG  ...    FG    FG%   3P  3P%    FT    FT%     PFPG   TS%  USG%  \
2606  0.508197  ...  53.0  0.736  0.0  0.0  35.0  0.625  1.42623  0.73  11.5   

      OWS  
2606  0.7  

[1 rows x 21 columns]


In [23]:
# Defining dataset of chosen stats
nba_data2 = nba_data.iloc[:,keep_list]

nba_data2 = nba_data2.loc[nba_data2['MPG'] > 6]
nba_data2.head(5)

for j in range(4,8):

# for j in range(4,len(nba_data2.columns)):
# j = 21

    output = nba_data2.columns[j]

    # df.loc[df['column_name'] == some_value]

    ### Building datasets and dropping "Season"

    # Reserving the 2018 season for "production"
    # ***Player name included
    nba_data_prod = nba_data2[nba_data2['Season'] == 2018]
    nba_data_prod = nba_data_prod.drop(columns=['Season'])

    # Setting the remainder of data for test and train
    nba_data_train_test = nba_data2[nba_data2['Season'] != 2018].drop(columns=["Player","Season"])

    # renaming for ease
    nba_tt = nba_data_train_test
    nba_tt.head(1)

    ### BUilding X and y dataframe 

    # Defining inputs
    X = nba_tt.drop(columns=output)

    # Defining outputs
    y = nba_tt[[output]].values.reshape(-1,1)

    print(X.shape)
    print(y.shape)
    y.max()

    X

    ### Starting data-modeling process 

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    ### Scaling data 

    from sklearn.preprocessing import StandardScaler

    # Create a StandardScater model and fit it to the training data
    X_scaler = StandardScaler().fit(X_train)

    # Transform the training and testing data using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # from sklearn.linear_model import LinearRegression
    # model = LinearRegression()

    from sklearn.neural_network import MLPRegressor
    model = MLPRegressor()

    ### Model fit and score 

    model.fit(X_train_scaled, y_train)
    training_score = model.score(X_train_scaled, y_train)
    testing_score = model.score(X_test_scaled, y_test)

    print(f"Training Score: {training_score}")
    print(f"Testing Score: {testing_score}")



    ### Running the Loop

    max_mpg = int(nba_tt["MPG"].max())
    PER_list = list()

    # Building predictions based on each minute of MPG
    for i in range(1,max_mpg+1):
        X_player["MPG"] = i
        X_player_scaled = X_scaler.transform(X_player)
        y_player = model.predict(X_player_scaled)

        PER_list.append(y_player[0])

    # PER_list

    # ### Summarizing Weights for each Stat

    # importance = model.coefs_
    # importance
    # # consider permutation feature importance 
    # # https://scikit-learn.org/stable/modules/permutation_importance.html


    # list1 = list()
    # list2 = list()

    # for i in range(0,len(X.columns)):
    #     list1.append(importance[0][i].round(4))
    #     list2.append(X.columns[i])

    # df = pd.DataFrame(list(zip(list1, list2)),
    #                columns =['Weight', 'Stat']).set_index('Stat').sort_values(by='Weight',ascending=False)
    # df.T

    ###  Summarizing PER results for each potential MPG

    import matplotlib.pyplot as plt

    plt.scatter(range(1,max_mpg+1), PER_list, c="blue", label=f"{output} by Minute")
    plt.legend()
    plt.title(f"MPG vs. {output}")
    plt.savefig(fname=f"Figures/{output}")
    
    plt.clf()

(5001, 21)
(5001, 1)


C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Score: 0.26753739071168126
Testing Score: 0.13351647485462326
(5001, 21)
(5001, 1)


C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Score: 0.8999805549885319
Testing Score: 0.8804477972595118
(5001, 21)
(5001, 1)


C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Score: 0.8440890937767853
Testing Score: 0.8181294294947383
(5001, 21)
(5001, 1)


C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\dcrus\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training Score: 0.9964888272677578
Testing Score: 0.9948998963935748


<Figure size 432x288 with 0 Axes>